<div align="center">

# 基于 LangChain 智能体的 Medallion 架构数据管道
使用 GenAI 驱动的 ETL 编排构建现代数据湖仓

</div>

<div align="center">

## 单元格 1：安装依赖并重启 Python
本单元格用于安装数据管道所需的全部 Python 库。构建和运行智能体管道需要 langchain、langchain-community、langchain-openai、langgraph、playwright 和 Pillow。在 Databricks 笔记本中，dbutils.library.restartPython() 至关重要，可确保新安装的库立即可用。

</div>

In [ ]:
%pip install -U \
  langchain==0.3.27 \
  langchain-community==0.3.27 \
  langchain-openai>=0.2.0 \
  langgraph==0.6.6 \
  Pillow==10.4.0

<div align="center">

## 单元格 2：配置 LangChain 与 LangSmith
本单元格设置 LangChain 和 LangSmith 的环境变量。LangSmith 用于对 LangChain 应用进行追踪、监控与调试。需配置端点、API 密钥（请替换为您的实际密钥）以及用于组织运行记录的项目名称。

</div>

In [ ]:
# Azure OpenAI
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://lixuesenuseno2.openai.azure.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "LXS-gpt-5"
os.environ["AZURE_OPENAI_API_VERSION"] = "2025-03-01-preview"
os.environ["AZURE_OPENAI_API_KEY"] = os.environ.get("AZURE_OPENAI_API_KEY", "XX")  # 建议通过环境变量注入真实密钥

print(f"Azure OpenAI endpoint: {os.environ['AZURE_OPENAI_ENDPOINT']}, deployment: {os.environ['AZURE_OPENAI_DEPLOYMENT_NAME']}")

<div align="center">

## 单元格 3：导入依赖
通过完整的库导入搭建基础环境，包括：
- LangChain 与 Databricks 的集成，用于基于 LLM 的智能体
- LangGraph 组件，用于编排智能体工作流
- PySpark SQL 能力，用于可扩展的数据转换
- 核心数据科学库，用于分析与可视化

</div>

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

from typing import Annotated, TypedDict, List, Optional
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, lit, md5, concat_ws, expr, to_date, upper, sum as _sum, avg as _avg, count as _count, date_format, year, month
from pyspark.sql.window import Window
from pyspark.sql.types import *
import json
import time
from datetime import datetime
import random
from IPython.display import Image, display

<div align="center">

## 单元格 4：初始化 Spark 会话、LLM 与数据库 Schema

本单元格初始化 SparkSession、用于智能体操作的大语言模型（LLM），并按照 Medallion 架构模式创建所需的数据库 Schema（Bronze、Silver、Gold）。同时在此定义 log_event 工具函数，用于在管道执行过程中输出结构化、带颜色的日志。

</div>

In [ ]:
def log_event(level, source, message):
    colors = {"AGENT": "\033[94m", "TOOL": "\033[93m", "INFO": "\033[92m", "ERROR": "\033[91m", "GRAPH": "\033[95m"}
    reset_color = "\033[0m"
    timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
    print(f"{timestamp} | {colors.get(level, '')}{level:<7}{reset_color} | {colors.get(level, '')}[{source}]{reset_color} {message}")

spark = SparkSession.builder.appName("AgenticDataPipeline").getOrCreate()

# llm = AzureChatOpenAI(
#     azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://lixuesenuseno2.openai.azure.com/"),
#     api_key=os.environ.get("AZURE_OPENAI_API_KEY", "XX"),
#     api_version="2025-01-01-preview",
#     azure_deployment="gpt-4.1",
#     temperature=0.0,
#     max_tokens=8000,
# )

llm = AzureChatOpenAI(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://lixuesenuseno2.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY", "XX"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2025-03-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "LXS-gpt-5"),
    max_completion_tokens=8000,
)

spark.sql("CREATE SCHEMA IF NOT EXISTS ops_bronze")
spark.sql("CREATE SCHEMA IF NOT EXISTS ops_silver")
spark.sql("CREATE SCHEMA IF NOT EXISTS ops_gold")

log_event("INFO", "Setup", "Environment initialized with Azure OpenAI (LXS-gpt-5)")

<div align="center">

## 单元格 5：生成 Bronze 层原始数据

本单元格为 Bronze 层创建合成原始数据，在 ops_bronze schema 下生成 customers_raw、transactions_raw、accounts_raw 和 opportunities_raw 表的数据。该步骤模拟将原始、可能杂乱的源数据摄入数据湖的过程。

</div>

In [ ]:
def create_bronze_data():
    log_event("INFO", "DataGen", "正在生成 Bronze 层数据...")
    
    customer_data = []
    for i in range(1, 101):
        name = f"FName{i} LName{i}" if random.random() > 0.1 else None
        email = f"fname{i}.lname{i}@email.com" if random.random() > 0.1 else "invalid-email"
        address = f"{i*10} Main St" if random.random() > 0.05 else None
        customer_data.append((i, name, email, address, f"2023-01-{random.randint(10, 28)}"))
    
    customer_data.extend([
        (1, 'John Doe', 'john.doe@email.com', '123 Elm St', '2023-01-15'), 
        (3, 'Peter Jones', 'peter.jones.dup@email.com', '789 Pine Ln', '2023-01-17')
    ])
    spark.createDataFrame(customer_data, ["customer_id", "name", "email", "address", "join_date"]).write.mode("overwrite").saveAsTable("ops_bronze.customers_raw")

    transactions_data = []
    for i in range(1, 201):
        cust_id = random.randint(1, 110)
        qty = random.randint(-5, 50)
        amount = f"${random.uniform(5, 1000):.2f}" if random.random() > 0.05 else None
        transactions_data.append((100+i, cust_id, qty, amount, f"2023-02-{random.randint(1, 28)}"))
    spark.createDataFrame(transactions_data, ["transaction_id", "customer_id", "quantity", "amount", "transaction_date"]).write.mode("overwrite").saveAsTable("ops_bronze.transactions_raw")

    accounts_data = [('ACC{:03d}'.format(i), f'GlobalCorp-{i}', random.choice(['Technology', 'Healthcare', 'Finance', 'TECH', None]), random.choice(['USA', 'UK', 'Germany'])) for i in range(1, 51)]
    accounts_data.append(('ACC001', 'Global Corporation', 'Tech', 'USA'))
    spark.createDataFrame(accounts_data, ["account_id", "account_name", "industry", "region"]).write.mode("overwrite").saveAsTable("ops_bronze.accounts_raw")
    
    opportunities_data = [(f'OPP{i:03d}', f"ACC{random.randint(1, 55):03d}", random.randint(-1000, 200000), '2024-07-15', random.choice(['Closed Won', 'Negotiation', 'Proposal', 'Qualification', 'Closed Lost'])) for i in range(1, 151)]
    spark.createDataFrame(opportunities_data, ["opportunity_id", "account_id", "value", "close_date", "stage"]).write.mode("overwrite").saveAsTable("ops_bronze.opportunities_raw")
    
    log_event("INFO", "DataGen", "Bronze data creation complete")
    display(spark.sql("SHOW TABLES IN ops_bronze"))

create_bronze_data()

<div align="center">

## 单元格 6：定义智能体工具

本单元格定义 LangGraph 工作流中智能体可调用的工具函数：

- **get_table_info**：对指定表进行画像，提供 Schema、行数与数据预览。
- **execute_pyspark_code**：在 Spark 环境中安全执行生成的 PySpark 代码，并处理错误。
- **create_notebook_visualization**：在 Databricks 笔记本内直接生成并展示基础可视化。

这些工具使 LLM 智能体能够与数据及环境进行交互。

</div>

In [ ]:
@tool
def get_table_info(table_name: str) -> str:
    """对表进行画像：包含 Schema、统计信息与数据预览，用于转换规划。"""
    try:
        log_event("TOOL", "get_table_info", f"正在画像: {table_name}")
        df = spark.table(table_name)
        schema_str = "\n".join([f"- {field.name}: {str(field.dataType)}" for field in df.schema.fields])
        count = df.count()
        preview = df.limit(3).toPandas().to_string()
        return f"TABLE: {table_name}\nROW_COUNT: {count}\n\nSCHEMA:\n{schema_str}\n\nSAMPLE_DATA:\n{preview}"
    except Exception as e:
        return f"错误: {str(e)}"

@tool
def execute_pyspark_code(code: str) -> str:
    """在具备错误处理与校验的前提下执行 PySpark 转换代码。"""
    try:
        log_event("TOOL", "execute_pyspark_code", "正在执行转换...")
        exec_globals = {
            'spark': spark, 'col': col, 'regexp_replace': regexp_replace, 'when': when, 'lit': lit,
            'md5': md5, 'concat_ws': concat_ws, 'expr': expr, 'to_date': to_date, 'upper': upper,
            '_sum': _sum, '_avg': _avg, '_count': _count, 'date_format': date_format, 'year': year, 'month': month, 'Window': Window
        }
        exec(code, exec_globals)
        log_event("TOOL", "execute_pyspark_code", "执行成功")
        return json.dumps({"status": "success", "error": None})
    except Exception as e:
        error_msg = f"执行错误: {str(e)}"
        log_event("ERROR", "execute_pyspark_code", error_msg)
        return json.dumps({"status": "error", "error": error_msg})
        
@tool
def create_notebook_visualization(table_name: str, plot_type: str, x_col: str, y_col: str, title: str) -> str:
    """创建优化的仪表盘可视化（表格预览 + matplotlib 图），自动排序与限制数据以提升性能。"""
    try:
        log_event("TOOL", "create_visualization", f"正在创建: {title}")

        df = spark.table(table_name)

        # 保留你原来的逻辑：Top bar -> y desc limit 5，否则按 x 排序
        if plot_type == 'bar' and 'Top' in title:
            df_display = df.orderBy(col(y_col).desc()).limit(5)
        else:
            df_display = df.orderBy(col(x_col))

        print(f"\n=== DASHBOARD COMPONENT: {title} ===")

        # 1) 表格预览（原逻辑不变）
        display(df_display)

        # 2) 真正画图（新增，但不改变结果表，只是多一个可视化输出）
        pdf = df_display.select(col(x_col).alias("x"), col(y_col).alias("y")).toPandas()

        import matplotlib.pyplot as plt

        # x 轴转成字符串，避免 category/日期等类型导致画图异常
        pdf["x"] = pdf["x"].astype(str)

        plt.figure()
        if plot_type == "bar":
            plt.bar(pdf["x"], pdf["y"])
            plt.xticks(rotation=30, ha="right")
        elif plot_type == "line":
            plt.plot(pdf["x"], pdf["y"], marker="o")
            plt.xticks(rotation=30, ha="right")
        else:
            return f"VISUALIZATION_ERROR: Unsupported plot_type='{plot_type}'. Use 'bar' or 'line'."

        plt.title(title)
        plt.xlabel(x_col)
        plt.ylabel(y_col)
        plt.tight_layout()
        plt.show()

        return f"SUCCESS: '{title}' visualization created and displayed"

    except Exception as e:
        return f"VISUALIZATION_ERROR: {str(e)}"

all_tools = [get_table_info, execute_pyspark_code, create_notebook_visualization]
tool_node = ToolNode(all_tools)

<div align="center">

## 单元格 7：定义智能体状态、节点与图工作流

这是智能体管道的核心部分。

### AgentState（智能体状态）
定义在 LangGraph 工作流中在智能体之间传递的共享状态对象，包括消息、当前任务、PySpark 代码、审查反馈、执行错误与重试次数。

### 智能体函数

- **planner_agent**：生成详细的数据转换方案
- **code_generator_agent**：根据方案与反馈编写 PySpark 代码
- **code_reviewer_agent**：对生成的代码进行质量与正确性审查
- **prepare_for_execution**：将 PySpark 代码格式化为可供工具执行的形式

### 条件边
- **after_review_decider**：根据代码审查反馈进行路由
- **after_execution_decider**：根据执行结果进行路由
二者共同实现自主自纠循环。

### 工作流定义
StateGraph 定义 Medallion 管道工作流的节点（智能体与工具）与边（状态转移）。

</div>

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list, lambda x, y: x + y]
    current_task: str
    pyspark_code: Optional[str]
    review_feedback: Optional[str]
    execution_error: Optional[str]
    retry_count: int

def planner_agent(state: AgentState):
    log_event("AGENT", "Planner", "正在制定转换策略")
    
    system_prompt = """You are an expert data architect specializing in Databricks Medallion architecture. 
    
    OBJECTIVES:
    - Create precise, executable PySpark transformation plans
    - Ensure data quality, deduplication, and proper type casting
    - Follow medallion best practices (Bronze=raw, Silver=cleaned, Gold=aggregated)
    - Design for scalability and performance optimization
    
    OUTPUT FORMAT:
    Provide a detailed, step-by-step technical plan with:
    1. Data profiling requirements
    2. Specific transformation logic
    3. Quality checks and validation rules
    4. Performance optimization strategies"""
    
    user_prompt = f"""转换任务: {state['current_task']}
    
    请制定完整的技术方案，涵盖：
    - 数据质量问题与修复
    - Schema 标准化与类型转换
    - 去重策略
    - 业务规则实现
    - 性能优化手段"""
    
    messages = [HumanMessage(content=f"{system_prompt}\n\n{user_prompt}")]
    response = llm.invoke(messages)
    return {"messages": [AIMessage(content=response.content, name="PlannerAgent")]}

def code_generator_agent(state: AgentState):
    log_event("AGENT", "CodeGenerator", "正在生成 PySpark 实现")

    # 取 Planner 的方案（如果没有也要容错）
    plan = next(
        (msg.content for msg in state.get("messages", []) 
         if isinstance(msg, AIMessage) and getattr(msg, "name", "") == "PlannerAgent"),
        ""
    )

    # 拼接上下文：优先采纳 review feedback，其次修复 execution error
    context_prompt = ""
    if state.get("review_feedback") and "APPROVED" not in (state["review_feedback"] or "").upper():
        context_prompt = f"\n\nREVIEW_FEEDBACK_TO_APPLY:\n{state['review_feedback']}"
    elif state.get("execution_error"):
        context_prompt = f"\n\nEXECUTION_ERROR_TO_FIX:\n{state['execution_error']}"

    system_prompt = """You are a senior PySpark developer building Databricks Medallion transformations.

HARD REQUIREMENTS (must follow exactly):
1) OUTPUT ONLY executable Python code. No markdown fences, no comments, no explanations.
2) The SparkSession variable `spark` is already available.
3) Read from Bronze tables and write to Silver tables using:
   df.write.mode("overwrite").saveAsTable("<target_table>")
4) Include explicit imports for every PySpark function you use (from pyspark.sql.functions / types / window).
5) Code must be deterministic and runnable on Databricks Serverless compute.

SERVERLESS CONSTRAINTS (critical):
- DO NOT use Spark SQL table caching or persistence commands:
  CACHE TABLE, UNCACHE TABLE, PERSIST TABLE, CLEAR CACHE
- DO NOT use spark.catalog.cacheTable / uncacheTable
- Avoid any SQL statements that may translate to table-level persistence.

PERFORMANCE (allowed optimizations that DO NOT change results):
- Use column pruning, filter pushdown, select only needed columns
- Avoid wide shuffles when not necessary
- Use DataFrame operations; if caching is considered, skip it (do not cache tables)

ROBUSTNESS:
- Handle nulls and malformed values safely
- Ensure type casting is safe (use when/otherwise, regexp, to_date, etc.)
- Deduplicate using Window + row_number or dropDuplicates with deterministic ordering
- If a column required by logic is missing, fail with a clear exception message.
- Before writing code, infer column names from the task text. NEVER invent required columns.
- For opportunities_raw, the numeric field is `value`. Create `opportunity_value` via withColumn, do not require it to exist.

OUTPUT CONTRACT:
- Must create exactly one target Silver table for the given task.
- Must not create temp views that replace the target contract.
"""

    user_prompt = f"""TASK:
{state.get('current_task', '')}

IMPLEMENTATION PLAN:
{plan}{context_prompt}

Generate the complete PySpark code that implements the task exactly.
Remember: output ONLY Python code, no markdown, no commentary.
"""

    messages = [HumanMessage(content=f"{system_prompt}\n\n{user_prompt}")]
    response = llm.invoke(messages)

    # 清理模型可能输出的代码块围栏（兜底）
    clean_code = (
        response.content.strip()
        .replace("```python", "")
        .replace("```", "")
        .strip()
    )

    return {
        "pyspark_code": clean_code,
        "execution_error": None,
        "review_feedback": None
    }

def code_reviewer_agent(state: AgentState):
    log_event("AGENT", "CodeReviewer", "正在进行质量审查")

    plan = next(
        (msg.content for msg in state.get("messages", [])
         if isinstance(msg, AIMessage) and getattr(msg, "name", "") == "PlannerAgent"),
        ""
    )

    system_prompt = """You are a senior Data Engineering QA reviewer for PySpark on Databricks.

You must strictly enforce correctness and Serverless compatibility.

REVIEW CHECKLIST (fail if any item is violated):
A) Output is pure Python code (no markdown fences, no explanations).
B) Uses SparkSession `spark` (assume pre-initialized) and reads from the specified source table(s).
C) Writes EXACTLY one target Silver table using:
   .write.mode("overwrite").saveAsTable("<target_table>")
D) Includes explicit imports for every used PySpark function/type/window.
E) Implements all data-quality rules from the plan/task (dedup, type casting, filtering, validations, flags).
F) Serverless constraints:
   - Must NOT contain: CACHE TABLE, UNCACHE TABLE, PERSIST TABLE, CLEAR CACHE
   - Must NOT call spark.catalog.cacheTable / uncacheTable
G) No dangerous dynamic execution or notebook magic (no %sql, no dbutils.fs, no system calls).

RESPONSE FORMAT (must follow exactly):
- If ALL checks pass: respond with exactly: APPROVED
- Otherwise: respond with:
  REJECTION_REASON: <bullet list of specific actionable fixes>
"""

    user_prompt = f"""TASK:
{state.get('current_task', '')}

PLAN:
{plan}

CODE TO REVIEW:
{state.get('pyspark_code', '')}
"""

    messages = [HumanMessage(content=f"{system_prompt}\n\n{user_prompt}")]
    response = llm.invoke(messages)

    return {"review_feedback": response.content}


def prepare_for_execution(state: AgentState):
    log_event("AGENT", "ExecutorPrep", "准备执行代码")
    tool_call_message = AIMessage(
        content="", 
        tool_calls=[{
            'name': 'execute_pyspark_code', 
            'args': {'code': state.get('pyspark_code')}, 
            'id': f'exec_{datetime.now().isoformat()}'
        }]
    )
    return {"messages": [tool_call_message]}

def after_review_decider(state: AgentState):
    feedback = state.get('review_feedback', '')
    if "APPROVED" in feedback.upper():
        log_event("GRAPH", "Router", "代码已通过 → 执行")
        return "prepare_for_execution"
    else:
        log_event("GRAPH", "Router", f"代码未通过 → 修订（第 {state.get('retry_count', 0) + 1} 次尝试）")
        return "revise_code"

def after_execution_decider(state: AgentState):
    tool_result = _parse_last_tool_result(state) or {"status": "error", "error": "No tool result found"}

    if tool_result.get("status") == "success":
        log_event("GRAPH", "Router", "执行成功 → 完成")
        return "__end__"

    # 失败：最多重试 3 次（与原逻辑含义一致：retry_count < 3 继续）
    next_retry = state.get("retry_count", 0) + 1
    if next_retry <= 3:
        log_event("GRAPH", "Router", f"执行失败 → 重试（第 {next_retry} 次）")
        return "revise_code"

    log_event("ERROR", "Router", "已达最大重试次数 → 中止")
    return "__end__"

def _parse_last_tool_result(state: AgentState):
    """只解析 messages[-1]，避免把历史 tool error 误当成当前错误"""
    if not state.get("messages"):
        return None
    last = state["messages"][-1]
    content = getattr(last, "content", None)
    if not content:
        return None
    try:
        obj = json.loads(content)
        if isinstance(obj, dict) and "status" in obj:
            return obj
    except Exception:
        return None
    return None

def revise_code_node_fn(state: AgentState):
    update = {"retry_count": state.get("retry_count", 0) + 1}

    # 如果刚刚执行过工具（executor 的输出通常是 JSON），把 error 写入 state 让下一轮生成代码能用
    tool_result = _parse_last_tool_result(state)
    if tool_result and tool_result.get("status") == "error":
        update["execution_error"] = tool_result.get("error")

    return update

workflow = StateGraph(AgentState)
workflow.add_node("planner", planner_agent)
workflow.add_node("code_generator", code_generator_agent)
workflow.add_node("code_reviewer", code_reviewer_agent)
workflow.add_node("revise_code_node", revise_code_node_fn)
workflow.add_node("prepare_for_execution", prepare_for_execution)
workflow.add_node("executor", tool_node)

workflow.set_entry_point("planner")
workflow.add_edge("planner", "code_generator")
workflow.add_edge("code_generator", "code_reviewer")
workflow.add_conditional_edges("code_reviewer", after_review_decider, {
    "prepare_for_execution": "prepare_for_execution", 
    "revise_code": "revise_code_node"
})
workflow.add_edge("revise_code_node", "code_generator")
workflow.add_edge("prepare_for_execution", "executor")
workflow.add_conditional_edges("executor", after_execution_decider, {
    "revise_code": "revise_code_node",
    "__end__": END
})

app = workflow.compile()
log_event("INFO", "Setup", "Enhanced LangGraph pipeline compiled")

In [ ]:
def create_workflow_diagram():
    """当可视化不可用时，生成基于文本的工作流图作为备选"""
    diagram = """
    ┌─────────────┐
    │   PLANNER   │
    │   AGENT     │
    └─────┬───────┘
          │
          ▼
    ┌─────────────┐
    │    CODE     │
    │ GENERATOR   │
    └─────┬───────┘
          │
          ▼
    ┌─────────────┐
    │    CODE     │
    │  REVIEWER   │
    └─────┬───────┘
          │
          ▼
    ┌─────────────┐    ┌─────────────┐
    │  APPROVED?  │───►│   REVISE    │
    │             │    │    CODE     │
    └─────┬───────┘    └─────┬───────┘
          │                  │
          ▼                  │
    ┌─────────────┐          │
    │  EXECUTE    │◄─────────┘
    │    CODE     │
    └─────┬───────┘
          │
          ▼
    ┌─────────────┐
    │  SUCCESS?   │
    │             │
    └─────────────┘
    """
    print("=== LangGraph 工作流图 ===")
    print(diagram)

try:
    # Try to create visual graph with improved error handling
    try:
        # First try with mermaid text output (safer fallback)
        mermaid_code = app.get_graph().draw_mermaid()
        print("=== WORKFLOW MERMAID DIAGRAM ===")
        print(mermaid_code)
    except Exception as mermaid_error:
        log_event("INFO", "Visualization", f"Mermaid generation failed: {mermaid_error}")
        
    # Try PNG generation with better error handling
    try:
        from langgraph.graph.graph import CompiledGraph
        if hasattr(app.get_graph(), 'draw_ascii'):
            ascii_graph = app.get_graph().draw_ascii()
            print("=== WORKFLOW ASCII DIAGRAM ===")
            print(ascii_graph)
        else:
            create_workflow_diagram()
    except Exception as png_error:
        log_event("INFO", "Visualization", f"PNG visualization unavailable: {png_error}")
        create_workflow_diagram()
        
except Exception as e:
    log_event("INFO", "Visualization", "Using text-based workflow diagram")
    create_workflow_diagram()

<div align="center">

## 单元格 9：执行 Bronze 到 Silver 的转换

本单元格启动 Medallion 管道的第一个主要阶段：将 Bronze 层的原始数据转换为清洗、标准化后的 Silver 层。它会遍历预定义的转换任务列表，每个任务描述对特定表（客户、交易、账户、商机）进行数据清洗与校验的要求。execute_pipeline_task 函数为每个任务编排智能体工作流。

</div>

In [ ]:
def execute_pipeline_task(task_description):
    task_name = task_description.strip().splitlines()[0][:50]
    log_event("INFO", "Pipeline", f"正在执行: {task_name}")
    
    initial_state = {
        "messages": [HumanMessage(content="初始化转换管道")], 
        "current_task": task_description, 
        "retry_count": 0
    }
    
    try:
        for state_update in app.stream(initial_state, {"recursion_limit": 80}):
            node_name = list(state_update.keys())[0]
            log_event("GRAPH", "Flow", f"已完成: {node_name}")
        
        log_event("INFO", "Pipeline", f"已完成: {task_name}")
    except Exception as e:
        log_event("ERROR", "Pipeline", f"任务失败: {task_name} - {str(e)}")
    
    time.sleep(1)

log_event("INFO", "PIPELINE", "===== BRONZE → SILVER TRANSFORMATIONS =====")

bronze_to_silver_transformations = [
    """CUSTOMER DATA CLEANSING: ops_bronze.customers_raw → ops_silver.customers_cleaned
    
    REQUIREMENTS:
    - Remove duplicate customer_id records (keep first occurrence)  
    - Fill null 'name' values with 'Unknown Customer'
    - Validate email format using regex pattern '.+@.+\\..+' 
    - Set invalid emails to null
    - Convert join_date string to proper DateType
    - Filter out records with null email OR null address
    - Add data quality flags for tracking""",
    
    """TRANSACTION DATA STANDARDIZATION: ops_bronze.transactions_raw → ops_silver.transactions_cleaned
    
    REQUIREMENTS:
    - Deduplicate on transaction_id (keep first record)
    - Clean amount field: remove '$' prefix and convert to Decimal(10,2)
    - Filter out negative quantity transactions
    - Filter out null or zero amount transactions  
    - Restrict to customer_id <= 100 (valid customers only)
    - Convert transaction_date to DateType
    - Add calculated fields for analysis""",
    
    """ACCOUNT DATA NORMALIZATION: ops_bronze.accounts_raw → ops_silver.accounts_cleaned
    
    REQUIREMENTS:
    - Deduplicate by account_id, preserving first record
    - Standardize industry values to uppercase
    - Map 'TECH' industry to 'TECHNOLOGY' 
    - Replace null industry with 'NOT_SPECIFIED'
    - Validate account_id format consistency
    - Add data lineage tracking columns""",
    
    """OPPORTUNITY DATA VALIDATION: ops_bronze.opportunities_raw → ops_silver.opportunities_cleaned
    
    REQUIREMENTS:
    - Source schema columns are exactly: opportunity_id, account_id, value, close_date, stage
    - Use source column name `value` (DO NOT assume `opportunity_value` exists). If you create a new column, name it `opportunity_value`.
    - Cast opportunity value to Decimal(14,2) with proper handling
    - Filter out opportunities with value <= 0
    - Validate account_id matches pattern 'ACC###'
    - Convert close_date string to DateType
    - Standardize stage values with proper casing
    - Add business rule validation flags"""
]

for task in bronze_to_silver_transformations:
    execute_pipeline_task(task)

<div align="center">

## 单元格 10：执行 Silver 到 Gold 的聚合

本单元格执行 Medallion 管道的第二个主要阶段：将 Silver 层的数据聚合并丰富到 Gold 层。它定义并执行创建分析表（如 customer_spending、account_performance、monthly_sales_summary）的任务。这些聚合为商业智能与报表提供数据准备。

</div>

In [ ]:
log_event("INFO", "PIPELINE", "===== SILVER → GOLD AGGREGATIONS =====")

silver_to_gold_aggregations = [ 
    """CUSTOMER SPENDING ANALYTICS: Create ops_gold.customer_spending
    SOURCE SCHEMA NOTE:
     - ops_silver.customers_cleaned columns include: customer_id, name, email, address, join_date (and possibly others)
     - DO NOT require or reference audit fields (effective_ts, updated_at) unless explicitly present.
    REQUIREMENTS:
     - Join ops_silver.customers_cleaned with ops_silver.transactions_cleaned
     - Group by the `customer_id` and `name` columns of the `customers_cleaned` table.
     - Calculate total_spent (sum of amounts) with null handling
     - Calculate total_transactions (count of transactions)
     - Calculate average_transaction_value 
     - Add customer spending tier classification
     - Order by total_spent descending for performance
     """,      
    """SOURCE SCHEMA NOTE:
- ops_silver.accounts_cleaned columns (types):
  - account_id (string)
  - account_name (string)
  - industry (string)
  - lineage_source (string)
  - lineage_processed_at (timestamp)

- ops_silver.opportunities_cleaned columns (types):
  - opportunity_id (string)
  - account_id (string)
  - opportunity_value (decimal(14,2))
  - close_date (date)
  - stage (string)
  - is_valid_value (boolean)
  - is_valid_account_id (boolean)
  - is_valid_close_date (boolean)
  - is_valid_stage (boolean)

IMPORTANT CONSTRAINTS:
- Use opportunity_value as the numeric field for all sums (do NOT use `value`).
- Join key is account_id (string) between the two tables.
- stage may have inconsistent casing; normalize before comparisons for 'Closed Won' and closed-stage logic.
- Validation flags exist but are optional for metrics; do not require any additional audit columns not listed above.

ACCOUNT PERFORMANCE METRICS: Create ops_gold.account_performance 
REQUIREMENTS: 
- Join ops_silver.accounts_cleaned with ops_silver.opportunities_cleaned 
- Group by account_id, account_name, and industry 
- Calculate total_pipeline_value (sum of all opportunity values) 
- Calculate won_value (sum where stage = 'Closed Won') 
- Calculate win_rate as percentage of closed won vs total closed 
- Count open_opportunities (non-closed stages) 
- Add performance ranking within industry
""",
"""
MONTHLY SALES TRENDS: Create ops_gold.monthly_sales_summary
    
    REQUIREMENTS:
    - Source from ops_silver.transactions_cleaned
    - Extract year-month from transaction_date as 'YYYY-MM' format
    - Group by month period  
    - Calculate monthly_revenue (sum of amounts)
    - Calculate monthly_transaction_count
    - Calculate average_transaction_size per month
    - Add month-over-month growth calculations
    - Order chronologically for time series analysis
"""
]

for task in silver_to_gold_aggregations:
    execute_pipeline_task(task)

In [ ]:
log_event("INFO", "PIPELINE", "===== BUSINESS INTELLIGENCE DASHBOARD =====")

<div align="center">

## 单元格 12：生成并校验仪表盘可视化

本单元格利用 bi_agent（使用 Notebook_visualization 工具）生成关键商业智能仪表盘组件，随后对 Gold 层表进行最终数据校验，通过展示记录数与 Top N 行，确保管道输出符合预期。

</div>

In [ ]:
# dashboard_prompt = """你是商业智能（BI）专家。你必须使用 create_notebook_visualization 工具在 Databricks Notebook 中创建 3 个组件。
# 严格要求：
# - 你必须调用 create_notebook_visualization 恰好 3 次。
# - 不允许调用 get_table_info、execute_pyspark_code 或任何其他工具。
# - 每次调用必须提供完整参数：table_name, plot_type, x_col, y_col, title。
# - 只输出工具调用，不要输出解释性文字。

# 要创建的 3 个可视化组件如下（按顺序）：

# 1) 客户分析（柱状图，Top 5）：
# - table_name = ops_gold.customer_spending
# - plot_type = bar
# - x_col = name
# - y_col = total_spent
# - title = Top 5 Customers by Total Spent

# 2) 行业绩效（柱状图，按行业分布）：
# - table_name = ops_gold.account_performance
# - plot_type = bar
# - x_col = industry
# - y_col = total_pipeline_value
# - title = Pipeline Value by Industry

# 3) 收入趋势（折线图）：
# - table_name = ops_gold.monthly_sales_summary
# - plot_type = line
# - x_col = month
# - y_col = monthly_revenue
# - title = Monthly Revenue Trend
# """
# print([t.name for t in all_tools])
# bi_agent = llm.bind_tools(all_tools)
# dashboard_response = bi_agent.invoke(dashboard_prompt)

# if dashboard_response.tool_calls:
#     for tool_call in dashboard_response.tool_calls:
#         tool_function = {t.name: t for t in all_tools}[tool_call['name']]
#         result = tool_function.invoke(tool_call['args'])
#         log_event("INFO", "Dashboard", f"已创建可视化: {tool_call['args'].get('title', 'Unknown')}")
# else:
#     log_event("ERROR", "Dashboard", "未能生成仪表盘可视化")

# log_event("INFO", "VALIDATION", "===== 最终数据校验 =====")

# validation_tables = [
#     "ops_gold.customer_spending",
#     "ops_gold.account_performance", 
#     "ops_gold.monthly_sales_summary"
# ]

# for table in validation_tables:
#     try:
#         print(f"\n=== 最终校验: {table} ===")
#         df = spark.table(table)
#         print(f"Record Count: {df.count()}")
#         if "customer_spending" in table:
#             display(df.orderBy(col("total_spent").desc()).limit(10))
#         elif "account_performance" in table:
#             display(df.orderBy(col("total_pipeline_value").desc()).limit(10))
#         else:
#             display(df.orderBy("month").limit(12))
#     except Exception as e:
#         log_event("ERROR", "Validation", f"校验 {table} 失败: {str(e)}")

# log_event("INFO", "COMPLETION", "===== Medallion 管道执行完成 =====")


create_notebook_visualization.invoke({
  "table_name":"ops_gold.customer_spending",
  "plot_type":"bar",
  "x_col":"name",
  "y_col":"total_spent",
  "title":"Top 5 Customers by Total Spent"
})

create_notebook_visualization.invoke({
  "table_name":"ops_gold.account_performance",
  "plot_type":"bar",
  "x_col":"industry",
  "y_col":"total_pipeline_value",
  "title":"Pipeline Value by Industry"
})

create_notebook_visualization.invoke({
  "table_name":"ops_gold.monthly_sales_summary",
  "plot_type":"line",
  "x_col":"month",
  "y_col":"monthly_revenue",
  "title":"Monthly Revenue Trend"
})

In [ ]:
print("\n🎉 SUCCESS: Databricks Medallion Architecture pipeline completed successfully!")
print("✅ All Bronze → Silver → Gold transformations executed")
print("✅ Business Intelligence dashboard components created")
print("✅ Data quality validation completed")